# Devasy Patel
# Practical 9
## 20BCE057
## Date : 17/04/2023

Aim : Implement a Traditional Recommender System using the Collaborative Filtering technique. Evaluation of Recommender system using RMSE(Root mean square error). 

In [1]:
import pandas as pd
import numpy
import warnings
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import nltk
import os
import math
import numpy as np
from numpy.linalg import norm
from sklearn.metrics.pairwise import cosine_similarity
import random as rd
from sklearn.metrics import mean_squared_error,mean_absolute_error

# User-User Collabrative Filtering (Numerical) - Accuracy

In [2]:
data = pd.read_csv("temp1.csv",header=None)

users = ["U"+str(i) for i in range(1,len(data)+1)]
items = ["P"+str(i) for i in range(1,len(data.columns)+1)]
print(data)

   0  1  2  3  4
0  0  3  0  3  4
1  4  0  0  2  0
2  0  0  3  0  0
3  3  0  4  0  3
4  4  3  0  4  4


In [3]:
with warnings.catch_warnings():
    warnings.filterwarnings("ignore")
    for user in data.index:
        ind = data.loc[user,:]!=0
        data.loc[user,ind] -= data.loc[user,ind].mean()
    print(data)

          0         1         2         3         4
0  0.000000 -0.333333  0.000000 -0.333333  0.666667
1  1.000000  0.000000  0.000000 -1.000000  0.000000
2  0.000000  0.000000  0.000000  0.000000  0.000000
3 -0.333333  0.000000  0.666667  0.000000 -0.333333
4  0.250000 -0.750000  0.000000  0.250000  0.250000


In [4]:
cos_sim = pd.DataFrame(cosine_similarity(data,data))
cos_sim = cos_sim.applymap(lambda x: round(x,3))
print(cos_sim)

       0      1    2      3      4
0  1.000  0.289  0.0 -0.333  0.471
1  0.289  1.000  0.0 -0.289 -0.000
2  0.000  0.000  0.0  0.000  0.000
3 -0.333 -0.289  0.0  1.000 -0.236
4  0.471 -0.000  0.0 -0.236  1.000


In [5]:
def predict(user,product,data,N):
    sim_list = cos_sim[user]
    ind_list = sorted(range(len(sim_list)), key=lambda x: sim_list[x])[-(N+1):-1]
    A=0
    B=0
    for i in ind_list:
        A += sim_list[i] * data.loc[i,product]
        B += sim_list[i]
    if(B==0):
        return 0
    else:
        return A/B
       
N = 3    


users = data.index
no_user = len(users)

products = data.columns
no_products = len(products)

rmse_val  = []
mae_val = []
no_ite = 10;

for x in range(no_ite):
    users_range =  sorted(rd.sample(range(0,no_user),2))
    products_range =  sorted(rd.sample(range(0,no_products),2))
    print("\nUser Sample range : ",users_range)
    print("Products Sample range : ",products_range)
    print("Sample : ")
    print(data.iloc[range(users_range[0],users_range[1]+1),range(products_range[0],products_range[1]+1)])
    y_pred = []
    y_true = []

    for user in range(users_range[0],users_range[1]+1): 
        rated_products = data.iloc[user,:].index[data.iloc[user,:]!=0]
        for i in range(products_range[0],products_range[1]+1):
            if(i in rated_products):
                y_pred.append(predict(user,i,data,N))
                y_true.append(data.iloc[user,i])

    rmse = math.sqrt(mean_squared_error(y_true, y_pred))
    print("RMSE : "+str(rmse))
    mae = mean_absolute_error(y_true,y_pred)
    print("MAE : "+str(mae))
    rmse_val.append(rmse)
    mae_val.append(mae)
print("\n\n")
print("-"*50)
print("\nAverage RMSE : "+str(sum(rmse_val)/no_ite))
print("\nAverage MAE : "+str(sum(mae_val)/no_ite))


User Sample range :  [3, 4]
Products Sample range :  [0, 2]
Sample : 
          0     1         2
3 -0.333333  0.00  0.666667
4  0.250000 -0.75  0.000000
RMSE : 0.6467130392237664
MAE : 0.5823809523809523

User Sample range :  [1, 2]
Products Sample range :  [1, 4]
Sample : 
     1    2    3    4
1  0.0  0.0 -1.0  0.0
2  0.0  0.0  0.0  0.0
RMSE : 0.6666666666666665
MAE : 0.6666666666666665

User Sample range :  [2, 4]
Products Sample range :  [1, 2]
Sample : 
      1         2
2  0.00  0.000000
3  0.00  0.666667
4 -0.75  0.000000
RMSE : 0.5559026693385971
MAE : 0.5416666666666665

User Sample range :  [0, 4]
Products Sample range :  [2, 4]
Sample : 
          2         3         4
0  0.000000 -0.333333  0.666667
1  0.000000 -1.000000  0.000000
2  0.000000  0.000000  0.000000
3  0.666667  0.000000 -0.333333
4  0.000000  0.250000  0.250000
RMSE : 0.5175455889267379
MAE : 0.4855406373075546

User Sample range :  [0, 4]
Products Sample range :  [2, 3]
Sample : 
          2         3
0  0.

# Item - Item Collabrative Filtering (Numerical) - Accuracy

In [6]:
data = pd.read_csv("temp1.csv",header=None)

users = ["U"+str(i) for i in range(1,len(data)+1)]
items = ["P"+str(i) for i in range(1,len(data.columns)+1)]
print(data)

   0  1  2  3  4
0  0  3  0  3  4
1  4  0  0  2  0
2  0  0  3  0  0
3  3  0  4  0  3
4  4  3  0  4  4


In [7]:
with warnings.catch_warnings():
    warnings.filterwarnings("ignore")
    for product in data.columns:
        ind = data[product]!=0
        data[product][ind] -= data[product][ind].mean()
    print(data)

          0  1    2  3         4
0  0.000000  0  0.0  0  0.333333
1  0.333333  0  0.0 -1  0.000000
2  0.000000  0 -0.5  0  0.000000
3 -0.666667  0  0.5  0 -0.666667
4  0.333333  0  0.0  1  0.333333


In [8]:
cos_sim = pd.DataFrame(cosine_similarity(data.T,data.T))
cos_sim = cos_sim.applymap(lambda x: round(x,3))
print(cos_sim)

       0    1      2      3      4
0  1.000  0.0 -0.577 -0.000  0.833
1  0.000  0.0  0.000  0.000  0.000
2 -0.577  0.0  1.000  0.000 -0.577
3 -0.000  0.0  0.000  1.000  0.289
4  0.833  0.0 -0.577  0.289  1.000


In [9]:
def predict(user,product,data,N):
    sim_list = cos_sim[product]
    ind_list = sorted(range(len(sim_list)), key=lambda x: sim_list[x])[-(N+1):-1]
    A=0
    B=0
    for i in ind_list:
        A += sim_list[i] * data[user][i]
        B += sim_list[i]
    if(B==0):
        return 0
    else:
        return A/B
       
# N = int(input("Value of N : "))
N = 3
# user = data.index.get_loc((input("Enter User Name : ")))

users = data.index
no_user = len(users)

products = data.columns
no_products = len(products)

rmse_val  = []
mae_val = []
no_ite = 10;

for x in range(no_ite):
    users_range =  sorted(rd.sample(range(0,no_user),2))
    products_range =  sorted(rd.sample(range(0,no_products),2))
    print("\nUser Sample range : ",users_range)
    print("Products Sample range : ",products_range)
    print("Sample : ")
    print(data.iloc[range(users_range[0],users_range[1]+1),range(products_range[0],products_range[1]+1)])
    y_pred = []
    y_true = []

    for user in range(users_range[0],users_range[1]+1): 
        rated_products = data.iloc[user,:].index[data.iloc[user,:]!=0]
        for i in range(products_range[0],products_range[1]+1):
            if(i in rated_products):
                y_pred.append(predict(user,i,data,N))
                y_true.append(data.iloc[user,i])

    rmse = math.sqrt(mean_squared_error(y_true, y_pred))
    print("RMSE : "+str(rmse))
    mae = mean_absolute_error(y_true,y_pred)
    print("MAE : "+str(mae))
    rmse_val.append(rmse)
    mae_val.append(mae)
print("\n\n")
print("-"*50)
print("\nAverage RMSE : "+str(sum(rmse_val)/no_ite))
print("\nAverage MAE : "+str(sum(mae_val)/no_ite))


User Sample range :  [2, 3]
Products Sample range :  [0, 2]
Sample : 
          0  1    2
2  0.000000  0 -0.5
3 -0.666667  0  0.5
RMSE : 1.0452715401237098
MAE : 0.8888888888888888

User Sample range :  [0, 2]
Products Sample range :  [3, 4]
Sample : 
   3         4
0  0  0.333333
1 -1  0.000000
2  0  0.000000
RMSE : 0.7921729649046887
MAE : 0.7525252525252526

User Sample range :  [1, 3]
Products Sample range :  [2, 4]
Sample : 
     2  3         4
1  0.0 -1  0.000000
2 -0.5  0  0.000000
3  0.5  0 -0.666667
RMSE : 0.6972166887783963
MAE : 0.6666666666666666

User Sample range :  [2, 3]
Products Sample range :  [1, 3]
Sample : 
   1    2  3
2  0 -0.5  0
3  0  0.5  0
RMSE : 0.5
MAE : 0.5

User Sample range :  [2, 3]
Products Sample range :  [1, 2]
Sample : 
   1    2
2  0 -0.5
3  0  0.5
RMSE : 0.5
MAE : 0.5

User Sample range :  [0, 3]
Products Sample range :  [0, 3]
Sample : 
          0  1    2  3
0  0.000000  0  0.0  0
1  0.333333  0  0.0 -1
2  0.000000  0 -0.5  0
3 -0.666667  0  0.

# User-User Collabrative Filtering (Dataset) - Accuracy

In [10]:
data = pd.read_csv("rating.csv",names=["user","product","rating","time"])
data = data.pivot_table(columns="product",index="user",values="rating")
data = data.iloc[:5000,:1500]
print(data)

product                059400232X  089933623X  094339676X  1034385789  \
user                                                                    
A001944026UMZ8T3K5QH1         NaN         NaN         NaN         NaN   
A00766851QZZUBOVF4JFT         NaN         NaN         NaN         NaN   
A01255851ZO1U93P8RKGE         NaN         NaN         NaN         NaN   
A014623426J5CM7M12MBW         NaN         NaN         NaN         NaN   
A01580702BRW77PSJ9X34         NaN         NaN         NaN         NaN   
...                           ...         ...         ...         ...   
A1F81EE548EOA5                NaN         NaN         NaN         NaN   
A1F81UOEZWZ01A                NaN         NaN         NaN         NaN   
A1F83CPW24HZKP                NaN         NaN         NaN         NaN   
A1F898XZRR8C28                NaN         NaN         NaN         NaN   
A1F89D3TCO7OGE                NaN         NaN         NaN         NaN   

product                1039869017  1085901386  109

In [11]:
with warnings.catch_warnings():
    warnings.filterwarnings("ignore")
    for user in data.index:
        ind = data.loc[user,:]!=np.nan
        data.loc[user,ind] -= data.loc[user,ind].mean()
    data = data.fillna(0)
    print(data)

product                059400232X  089933623X  094339676X  1034385789  \
user                                                                    
A001944026UMZ8T3K5QH1         0.0         0.0         0.0         0.0   
A00766851QZZUBOVF4JFT         0.0         0.0         0.0         0.0   
A01255851ZO1U93P8RKGE         0.0         0.0         0.0         0.0   
A014623426J5CM7M12MBW         0.0         0.0         0.0         0.0   
A01580702BRW77PSJ9X34         0.0         0.0         0.0         0.0   
...                           ...         ...         ...         ...   
A1F81EE548EOA5                0.0         0.0         0.0         0.0   
A1F81UOEZWZ01A                0.0         0.0         0.0         0.0   
A1F83CPW24HZKP                0.0         0.0         0.0         0.0   
A1F898XZRR8C28                0.0         0.0         0.0         0.0   
A1F89D3TCO7OGE                0.0         0.0         0.0         0.0   

product                1039869017  1085901386  109

In [12]:
cos_sim = pd.DataFrame(cosine_similarity(data,data))
cos_sim = cos_sim.applymap(lambda x: round(x,3))
print(cos_sim)

      0     1     2     3     4     5     6     7     8     9     ...  4990  \
0      0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
1      0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
2      0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
3      0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
4      0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
...    ...   ...   ...   ...   ...   ...   ...   ...   ...   ...  ...   ...   
4995   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
4996   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
4997   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
4998   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
4999   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   

      4991  4992  4993  4994  4995  4996  4997  499

In [13]:
def predict(user,product,data,N):
    sim_list = cos_sim[user]
    ind_list = sorted(range(len(sim_list)), key=lambda x: sim_list[x])[-(N+1):-1]
    A=0
    B=0
    for i in ind_list:
        A += sim_list[i] * data.iloc[i,product]
        B += sim_list[i]
    if(B==0):
        return 0
    else:
        return A/B
       
# N = int(input("Value of N : "))
N = 20

users = data.index
no_user = len(users)

products = data.columns
no_products = len(products)

users_range =  sorted(rd.sample(range(0,no_user),2))
products_range =  sorted(rd.sample(range(0,no_products),2))


rmse_val = []
mae_val = []
no_ite = 10


for x in range(no_ite):

    print("\n\nUser Sample range : ",users_range)
    print("Products Sample range : ",products_range)
    print("Sample : ")
    print(data.iloc[range(users_range[0],users_range[1]+1),range(products_range[0],products_range[1]+1)])
    y_pred = []
    y_true = []

    for user in range(users_range[0],users_range[1]+1): 
        rated_products = data.iloc[user,:].index[data.iloc[user,:]!=0]
        for i in range(products_range[0],products_range[1]+1):
            if(i in rated_products):
                y_pred.append(predict(user,i,data,N))
                y_true.append(data.iloc[user,i])
        if(len(y_pred)>0 and len(y_pred)==len(y_true)):
            rmse = math.sqrt(mean_squared_error(y_true, y_pred))
            print("RMSE : "+str(rmse))
            mae = mean_absolute_error(y_true,y_pred)
            print("MAE : "+str(mae))
            rmse_val.append(rmse)
            mae_val.append(mae)
print("\n\n")
print("-"*50)
print("\nAverage RMSE : "+str(sum(rmse_val)/no_ite))
print("\nAverage MAE : "+str(sum(mae_val)/no_ite))



User Sample range :  [3049, 4466]
Products Sample range :  [700, 1172]
Sample : 
product         9983814129  998383054X  9983830884  9983833085  9983853027  \
user                                                                         
A1981BJUPA3R8C         0.0         0.0         0.0         0.0         0.0   
A1989944VHPI2I         0.0         0.0         0.0         0.0         0.0   
A1989WE0ZSBKNN         0.0         0.0         0.0         0.0         0.0   
A198A15LMKHG8T         0.0         0.0         0.0         0.0         0.0   
A198CPQ6A1VYD2         0.0         0.0         0.0         0.0         0.0   
...                    ...         ...         ...         ...         ...   
A1DLFC5GG1RQFL         0.0         0.0         0.0         0.0         0.0   
A1DLMTOTHQ4AST         0.0         0.0         0.0         0.0         0.0   
A1DLQENEM4ANWH         0.0         0.0         0.0         0.0         0.0   
A1DLSQMEALZSIK         0.0         0.0         0.0         



User Sample range :  [3049, 4466]
Products Sample range :  [700, 1172]
Sample : 
product         9983814129  998383054X  9983830884  9983833085  9983853027  \
user                                                                         
A1981BJUPA3R8C         0.0         0.0         0.0         0.0         0.0   
A1989944VHPI2I         0.0         0.0         0.0         0.0         0.0   
A1989WE0ZSBKNN         0.0         0.0         0.0         0.0         0.0   
A198A15LMKHG8T         0.0         0.0         0.0         0.0         0.0   
A198CPQ6A1VYD2         0.0         0.0         0.0         0.0         0.0   
...                    ...         ...         ...         ...         ...   
A1DLFC5GG1RQFL         0.0         0.0         0.0         0.0         0.0   
A1DLMTOTHQ4AST         0.0         0.0         0.0         0.0         0.0   
A1DLQENEM4ANWH         0.0         0.0         0.0         0.0         0.0   
A1DLSQMEALZSIK         0.0         0.0         0.0         



User Sample range :  [3049, 4466]
Products Sample range :  [700, 1172]
Sample : 
product         9983814129  998383054X  9983830884  9983833085  9983853027  \
user                                                                         
A1981BJUPA3R8C         0.0         0.0         0.0         0.0         0.0   
A1989944VHPI2I         0.0         0.0         0.0         0.0         0.0   
A1989WE0ZSBKNN         0.0         0.0         0.0         0.0         0.0   
A198A15LMKHG8T         0.0         0.0         0.0         0.0         0.0   
A198CPQ6A1VYD2         0.0         0.0         0.0         0.0         0.0   
...                    ...         ...         ...         ...         ...   
A1DLFC5GG1RQFL         0.0         0.0         0.0         0.0         0.0   
A1DLMTOTHQ4AST         0.0         0.0         0.0         0.0         0.0   
A1DLQENEM4ANWH         0.0         0.0         0.0         0.0         0.0   
A1DLSQMEALZSIK         0.0         0.0         0.0         



User Sample range :  [3049, 4466]
Products Sample range :  [700, 1172]
Sample : 
product         9983814129  998383054X  9983830884  9983833085  9983853027  \
user                                                                         
A1981BJUPA3R8C         0.0         0.0         0.0         0.0         0.0   
A1989944VHPI2I         0.0         0.0         0.0         0.0         0.0   
A1989WE0ZSBKNN         0.0         0.0         0.0         0.0         0.0   
A198A15LMKHG8T         0.0         0.0         0.0         0.0         0.0   
A198CPQ6A1VYD2         0.0         0.0         0.0         0.0         0.0   
...                    ...         ...         ...         ...         ...   
A1DLFC5GG1RQFL         0.0         0.0         0.0         0.0         0.0   
A1DLMTOTHQ4AST         0.0         0.0         0.0         0.0         0.0   
A1DLQENEM4ANWH         0.0         0.0         0.0         0.0         0.0   
A1DLSQMEALZSIK         0.0         0.0         0.0         



User Sample range :  [3049, 4466]
Products Sample range :  [700, 1172]
Sample : 
product         9983814129  998383054X  9983830884  9983833085  9983853027  \
user                                                                         
A1981BJUPA3R8C         0.0         0.0         0.0         0.0         0.0   
A1989944VHPI2I         0.0         0.0         0.0         0.0         0.0   
A1989WE0ZSBKNN         0.0         0.0         0.0         0.0         0.0   
A198A15LMKHG8T         0.0         0.0         0.0         0.0         0.0   
A198CPQ6A1VYD2         0.0         0.0         0.0         0.0         0.0   
...                    ...         ...         ...         ...         ...   
A1DLFC5GG1RQFL         0.0         0.0         0.0         0.0         0.0   
A1DLMTOTHQ4AST         0.0         0.0         0.0         0.0         0.0   
A1DLQENEM4ANWH         0.0         0.0         0.0         0.0         0.0   
A1DLSQMEALZSIK         0.0         0.0         0.0         




--------------------------------------------------

Average RMSE : 0.0

Average MAE : 0.0


# Item - Item Collabrative Filtering (Dataset)

In [14]:
data = pd.read_csv("rating.csv",names=["user","product","rating","time"])
data = data.pivot_table(columns="product",index="user",values="rating")
print(data)

product                059400232X  089933623X  094339676X  1034385789  \
user                                                                    
A001944026UMZ8T3K5QH1         NaN         NaN         NaN         NaN   
A00766851QZZUBOVF4JFT         NaN         NaN         NaN         NaN   
A01255851ZO1U93P8RKGE         NaN         NaN         NaN         NaN   
A014623426J5CM7M12MBW         NaN         NaN         NaN         NaN   
A01580702BRW77PSJ9X34         NaN         NaN         NaN         NaN   
...                           ...         ...         ...         ...   
AZZMV5VT9W7Y8                 NaN         NaN         NaN         NaN   
AZZP8BC9WE0N8                 NaN         NaN         NaN         NaN   
AZZST8OYL5P4Q                 NaN         NaN         NaN         NaN   
AZZUBI2MEAHI9                 NaN         NaN         NaN         NaN   
AZZZEWADNFNUM                 NaN         NaN         NaN         NaN   

product                1039869017  1085901386  109

In [15]:
with warnings.catch_warnings():
    warnings.filterwarnings("ignore")
    for product in data.columns:
        ind = data[product]!=np.nan
        data[product][ind] -= data[product][ind].mean()
    data = data.fillna(0)
    print(data)

product                059400232X  089933623X  094339676X  1034385789  \
user                                                                    
A001944026UMZ8T3K5QH1         0.0         0.0         0.0         0.0   
A00766851QZZUBOVF4JFT         0.0         0.0         0.0         0.0   
A01255851ZO1U93P8RKGE         0.0         0.0         0.0         0.0   
A014623426J5CM7M12MBW         0.0         0.0         0.0         0.0   
A01580702BRW77PSJ9X34         0.0         0.0         0.0         0.0   
...                           ...         ...         ...         ...   
AZZMV5VT9W7Y8                 0.0         0.0         0.0         0.0   
AZZP8BC9WE0N8                 0.0         0.0         0.0         0.0   
AZZST8OYL5P4Q                 0.0         0.0         0.0         0.0   
AZZUBI2MEAHI9                 0.0         0.0         0.0         0.0   
AZZZEWADNFNUM                 0.0         0.0         0.0         0.0   

product                1039869017  1085901386  109

In [16]:
cos_sim = pd.DataFrame(cosine_similarity(data.T,data.T))
cos_sim = cos_sim.applymap(lambda x: round(x,3))
print(cos_sim)

      0     1     2     3     4     5     6     7     8     9     ...   3323  \
0      0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...  0.000   
1      0.0   1.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...  0.000   
2      0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...  0.000   
3      0.0   0.0   0.0   1.0   0.0   0.0   0.0   0.0   0.0   0.0  ...  0.000   
4      0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...  0.000   
...    ...   ...   ...   ...   ...   ...   ...   ...   ...   ...  ...    ...   
3328   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ... -0.001   
3329   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...  0.000   
3330   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...  0.003   
3331   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...  0.000   
3332   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...  0.000   

       3324   3325   3326  3327   3328 

In [17]:
def predict(user,product,data,N):
    sim_list = cos_sim[product]
    ind_list = sorted(range(len(sim_list)), key=lambda x: sim_list[x])[-(N+1):-1]
    A=0
    B=0
    for i in ind_list:
        A += sim_list[i] * data.iloc[user,i]
        B += sim_list[i]
    if(B==0):
        return 0
    else:
        return A/B
       
# N = int(input("Value of N : "))
N = 20

users = data.index
no_user = len(users)

products = data.columns
no_products = len(products)

users_range =  sorted(rd.sample(range(0,no_user),2))
products_range =  sorted(rd.sample(range(0,no_products),2))


rmse_val = []
mae_val = []
no_ite = 10


for x in range(no_ite):

    print("\n\nUser Sample range : ",users_range)
    print("Products Sample range : ",products_range)
    print("Sample : ")
    print(data.iloc[range(users_range[0],users_range[1]+1),range(products_range[0],products_range[1]+1)])
    y_pred = []
    y_true = []

    for user in range(users_range[0],users_range[1]+1): 
        rated_products = data.iloc[user,:].index[data.iloc[user,:]!=0]
        for i in range(products_range[0],products_range[1]+1):
            if(i in rated_products):
                y_pred.append(predict(user,i,data,N))
                y_true.append(data.iloc[user,i])
        if(len(y_pred)>0 and len(y_pred)==len(y_true)):
            rmse = math.sqrt(mean_squared_error(y_true, y_pred))
            print("RMSE : "+str(rmse))
            mae = mean_absolute_error(y_true,y_pred)
            print("MAE : "+str(mae))
            rmse_val.append(rmse)
            mae_val.append(mae)
print("\n\n")
print("-"*50)
print("\nAverage RMSE : "+str(sum(rmse_val)/no_ite))
print("\nAverage MAE : "+str(sum(mae_val)/no_ite))



User Sample range :  [265, 16307]
Products Sample range :  [764, 2388]
Sample : 
product         9984976734  9984976866  9984977234  9984977595  9984977730  \
user                                                                         
A10R7XM20Z0323         0.0         0.0         0.0         0.0         0.0   
A10RDB0FTFA5S3         0.0         0.0         0.0         0.0         0.0   
A10RFZ7WJD1Y2C         0.0         0.0         0.0         0.0         0.0   
A10RHYA8B2K9QT         0.0         0.0         0.0         0.0         0.0   
A10RMMITRBG4W          0.0         0.0         0.0         0.0         0.0   
...                    ...         ...         ...         ...         ...   
A2E6WMRJAR02Q1         0.0         0.0         0.0         0.0         0.0   
A2E74YKJXGZCJB         0.0         0.0         0.0         0.0         0.0   
A2E76LB77DZVN2         0.0         0.0         0.0         0.0         0.0   
A2E7CEPDGY42VQ         0.0         0.0         0.0         



User Sample range :  [265, 16307]
Products Sample range :  [764, 2388]
Sample : 
product         9984976734  9984976866  9984977234  9984977595  9984977730  \
user                                                                         
A10R7XM20Z0323         0.0         0.0         0.0         0.0         0.0   
A10RDB0FTFA5S3         0.0         0.0         0.0         0.0         0.0   
A10RFZ7WJD1Y2C         0.0         0.0         0.0         0.0         0.0   
A10RHYA8B2K9QT         0.0         0.0         0.0         0.0         0.0   
A10RMMITRBG4W          0.0         0.0         0.0         0.0         0.0   
...                    ...         ...         ...         ...         ...   
A2E6WMRJAR02Q1         0.0         0.0         0.0         0.0         0.0   
A2E74YKJXGZCJB         0.0         0.0         0.0         0.0         0.0   
A2E76LB77DZVN2         0.0         0.0         0.0         0.0         0.0   
A2E7CEPDGY42VQ         0.0         0.0         0.0         



User Sample range :  [265, 16307]
Products Sample range :  [764, 2388]
Sample : 
product         9984976734  9984976866  9984977234  9984977595  9984977730  \
user                                                                         
A10R7XM20Z0323         0.0         0.0         0.0         0.0         0.0   
A10RDB0FTFA5S3         0.0         0.0         0.0         0.0         0.0   
A10RFZ7WJD1Y2C         0.0         0.0         0.0         0.0         0.0   
A10RHYA8B2K9QT         0.0         0.0         0.0         0.0         0.0   
A10RMMITRBG4W          0.0         0.0         0.0         0.0         0.0   
...                    ...         ...         ...         ...         ...   
A2E6WMRJAR02Q1         0.0         0.0         0.0         0.0         0.0   
A2E74YKJXGZCJB         0.0         0.0         0.0         0.0         0.0   
A2E76LB77DZVN2         0.0         0.0         0.0         0.0         0.0   
A2E7CEPDGY42VQ         0.0         0.0         0.0         



User Sample range :  [265, 16307]
Products Sample range :  [764, 2388]
Sample : 
product         9984976734  9984976866  9984977234  9984977595  9984977730  \
user                                                                         
A10R7XM20Z0323         0.0         0.0         0.0         0.0         0.0   
A10RDB0FTFA5S3         0.0         0.0         0.0         0.0         0.0   
A10RFZ7WJD1Y2C         0.0         0.0         0.0         0.0         0.0   
A10RHYA8B2K9QT         0.0         0.0         0.0         0.0         0.0   
A10RMMITRBG4W          0.0         0.0         0.0         0.0         0.0   
...                    ...         ...         ...         ...         ...   
A2E6WMRJAR02Q1         0.0         0.0         0.0         0.0         0.0   
A2E74YKJXGZCJB         0.0         0.0         0.0         0.0         0.0   
A2E76LB77DZVN2         0.0         0.0         0.0         0.0         0.0   
A2E7CEPDGY42VQ         0.0         0.0         0.0         



User Sample range :  [265, 16307]
Products Sample range :  [764, 2388]
Sample : 
product         9984976734  9984976866  9984977234  9984977595  9984977730  \
user                                                                         
A10R7XM20Z0323         0.0         0.0         0.0         0.0         0.0   
A10RDB0FTFA5S3         0.0         0.0         0.0         0.0         0.0   
A10RFZ7WJD1Y2C         0.0         0.0         0.0         0.0         0.0   
A10RHYA8B2K9QT         0.0         0.0         0.0         0.0         0.0   
A10RMMITRBG4W          0.0         0.0         0.0         0.0         0.0   
...                    ...         ...         ...         ...         ...   
A2E6WMRJAR02Q1         0.0         0.0         0.0         0.0         0.0   
A2E74YKJXGZCJB         0.0         0.0         0.0         0.0         0.0   
A2E76LB77DZVN2         0.0         0.0         0.0         0.0         0.0   
A2E7CEPDGY42VQ         0.0         0.0         0.0         




--------------------------------------------------

Average RMSE : 0.0

Average MAE : 0.0
